In [2]:
import pandas as pd
import numpy as np
import re

#googlee play scrapper
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

#cleaning
import contractions #can't -> cannot
import string #hi! -> hi
from cleantext import clean #emoji

#detect language
from langdetect import detect

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
pd.reset_option('display.max_rows')

# alodokter

In [14]:
reviews = reviews_all(
    'com.alodokter.android',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

alodokterraw = pd.DataFrame(np.array(reviews),columns=['review'])
alodokterraw = alodokterraw.join(pd.DataFrame(alodokterraw.pop('review').tolist()))
alodokterraw['at'] = pd.to_datetime(alodokterraw['at'], format='%Y-%m-%d')
alodokterraw = alodokterraw.loc[(alodokterraw['at'] <= "2023-5-31")]
alodokterraw

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
51,4bd80f92-b673-40ee-b57c-f15e1e17ccea,Cetta Adhipurusa,https://play-lh.googleusercontent.com/a-/AD_cM...,bagus,5,0,5.5.0,2023-05-30 13:48:58,None,NaT
52,3f0e1849-dacb-4207-a2f0-81662e299257,HeLen WeLL,https://play-lh.googleusercontent.com/a/AAcHTt...,Saya merasa terbantu dengan aplikasi ini dan s...,4,0,5.5.0,2023-05-27 17:02:45,None,NaT
53,cf9ae3ce-6970-4d58-b181-4c104d9bb964,Yosua Reinaldo,https://play-lh.googleusercontent.com/a/AAcHTt...,Komentar nya cukup standar sesuai google,4,0,5.5.0,2023-05-24 22:09:53,None,NaT
54,ca5304d9-3152-4424-a235-040e21cc5c7b,reni setiawati,https://play-lh.googleusercontent.com/a-/AD_cM...,sangat membantu,5,0,5.5.0,2023-05-21 21:10:54,None,NaT
55,0c1f5fa7-5623-4d16-8673-82d5e1f2ba7d,Gusti Mustikawati,https://play-lh.googleusercontent.com/a-/AD_cM...,Lemot bangeettttt apps nyaaaaaa,1,0,None,2023-05-19 18:25:36,"Alo, Gusti Mustikawati! Mohon maaf atas ketida...",2023-05-22 09:11:32
...,...,...,...,...,...,...,...,...,...,...
18536,f6f1fbd8-c7e7-4533-af46-91a47ff972e3,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"I love it, this app makes me easier to consult...",5,2,None,2016-03-15 15:15:36,None,NaT
18537,041f19c5-4ab9-49ef-af67-6538e5f9b7b6,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Finally alodokter launch the alodokter apps in...,5,0,1.0.0,2016-03-14 19:25:34,None,NaT
18538,309e6e74-ad6a-4660-8dfe-bb4bf297a999,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Sangat berguna untuk zaman sekarang yang diman...,5,0,None,2016-03-14 10:53:01,None,NaT
18539,c567eac0-6b7c-4d6c-a1f8-9b369618970e,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Berguna untuk menjawab pertanyaan saya tentang...,5,1,None,2016-03-14 10:36:42,None,NaT


In [15]:
alodokterraw.to_csv('alodokterraw.csv',index=False)

In [7]:
alodokterfiltered = alodokterraw.copy()
alodokterfiltered = alodokterfiltered.reset_index(drop=True)
alodokterfiltered = alodokterfiltered.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','score'])
alodokterfiltered = alodokterfiltered[[len(i.split())>2 for i in alodokterfiltered['content'].values]]
alodokterfiltered.to_csv('alodokterfiltered.csv',index=False)
alodokterfiltered

,content
1,Saya merasa terbantu dengan aplikasi ini dan s...
2,Komentar nya cukup standar sesuai google
4,Lemot bangeettttt apps nyaaaaaa
6,Cukup membantu kalo lg males ke rumah sakit cu...
7,"Dokternya cepat respon dan baik, sangat helpfu..."
...,...
18485,"I love it, this app makes me easier to consult..."
18486,Finally alodokter launch the alodokter apps in...
18487,Sangat berguna untuk zaman sekarang yang diman...
18488,Berguna untuk menjawab pertanyaan saya tentang...


In [5]:
alodokterlang = pd.read_csv(r'alodokterfiltered.csv')

pattern = r'\bapp(s)?\b'
replacement = 'application'

def replace_applications(text):
    return re.sub(pattern, replacement, text, flags=re.IGNORECASE)

alodokterlang['clean_emoji'] = alodokterlang['content'].apply(lambda x: clean(x, no_emoji=True))
alodokterlang['clean_emoji'] = alodokterlang['clean_emoji'].apply(replace_applications)
alodokterlang['clean_emoji'] = alodokterlang['clean_emoji'].apply(lambda x: contractions.fix(x))

alodokter_languages = []

for index, row in alodokterlang.iterrows():
    try:
        language = detect(row['clean_emoji'])
    except:
        language = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['language'] = language
    alodokter_languages.append(row)

alodokterlang = pd.DataFrame(alodokter_languages)

#print(alodokterlang)

alodokterlang['language'].value_counts()

id       7720
en       2309
tl        152
no         59
fr         57
af         45
so         41
ca         40
nl         35
it         32
sl         29
sq         22
et         21
da         21
cy         16
hr         15
ro         13
sv         11
lt         10
es          9
cs          7
sk          6
pt          6
fi          4
tr          4
hu          4
pl          4
sw          3
lv          2
error       1
Name: language, dtype: int64

In [20]:
alodokterlang.to_csv('alodokterlang.csv',index=False)

# klikdokter

In [17]:
reviews = reviews_all(
    'id.codigo.klikdokter',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

klikdokterraw = pd.DataFrame(np.array(reviews),columns=['review'])
klikdokterraw = klikdokterraw.join(pd.DataFrame(klikdokterraw.pop('review').tolist()))
klikdokterraw['at'] = pd.to_datetime(klikdokterraw['at'], format='%Y-%m-%d')
klikdokterraw = klikdokterraw.loc[(klikdokterraw['at'] <= "2023-5-31")]
klikdokterraw

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
2,a74fcbb7-8da5-4baf-84c1-fb105a558d78,Meilisa Puspita Handayani,https://play-lh.googleusercontent.com/a-/AD_cM...,"Gagal menghubungkan ke dokter, uang direfund d...",1,0,2.1.36,2023-04-06 20:57:25,"Dear Meilisa Puspita Handayani, mohon maaf ata...",2023-04-06 22:06:38
3,a305fd8e-d33a-40d2-86dd-e52440548954,Rayfan Tio Saputro,https://play-lh.googleusercontent.com/a/AAcHTt...,Konsep refundnya kok berupa voucher,1,0,2.1.36,2023-03-31 17:49:58,"Dear Rayfan Tio Saputro, mohon maaf atas ketid...",2023-03-31 18:16:44
4,815921fc-3149-429a-8b5d-22ffba6c0cd9,sheila sofie,https://play-lh.googleusercontent.com/a-/AD_cM...,Thank you,5,0,None,2023-03-20 04:37:46,"Dear sheila sofie, terima kasih sudah mengguna...",2023-03-20 04:41:46
5,0a9450d9-ec77-4012-a8c0-d2258177b3ea,Anthonio Wirman L,https://play-lh.googleusercontent.com/a-/AD_cM...,ini app nya kenapa belakangan selalu muncul me...,5,0,2.1.33,2023-02-14 22:25:16,"Hai Anthonio Wirman L, terima kasih atas saran...",2021-11-09 14:05:53
6,b4b33219-edbb-4109-99a2-653059a25236,Cindy Fitriani,https://play-lh.googleusercontent.com/a/AAcHTt...,Aplikasi gak jelas gak usaj di instal,1,0,2.1.33,2023-02-13 09:59:11,"Dear Cindy Fitriani, mohon maaf atas ketidakny...",2023-02-13 10:23:44
...,...,...,...,...,...,...,...,...,...,...
789,f84817a7-6c0f-401e-98d9-c4f9ebafeee7,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Bagus banget.. sangat bermanfaat dan mudah dig...,5,0,1.0,2015-02-19 08:37:39,None,NaT
790,8513e4d9-b272-40d5-bf2d-b7a2dfe2a15b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Membantu banget buat cari-cari info kesehatan!,5,0,1.0,2015-02-19 08:30:38,None,NaT
791,7ba5bfac-41e0-4bf6-9df6-831b19bda6e4,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Very useful, complete info...thanks Dok",5,0,1.0,2015-02-19 01:10:33,None,NaT
792,22fd5383-0f8c-4360-9963-7e0cb881e025,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I love the Tanya Dokter feature. Informative a...,5,0,1.0,2015-02-18 23:27:28,None,NaT


In [18]:
klikdokterraw.to_csv('klikdokterraw.csv',index=False)

In [32]:
klikdokterfiltered = klikdokterraw.copy()
klikdokterfiltered = klikdokterfiltered.reset_index(drop=True)
klikdokterfiltered = klikdokterfiltered.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','score'])
klikdokterfiltered = klikdokterfiltered[[len(i.split())>2 for i in klikdokterfiltered['content'].values]]
klikdokterfiltered.to_csv('klikdokterfiltered.csv',index=False)
klikdokterfiltered

,content
0,"Gagal menghubungkan ke dokter, uang direfund d..."
1,Konsep refundnya kok berupa voucher
3,ini app nya kenapa belakangan selalu muncul me...
4,Aplikasi gak jelas gak usaj di instal
5,Nyoba untuk konsul ke dokter. Tapi udah nyari2...
...,...
787,Bagus banget.. sangat bermanfaat dan mudah dig...
788,Membantu banget buat cari-cari info kesehatan!
789,"Very useful, complete info...thanks Dok"
790,I love the Tanya Dokter feature. Informative a...


In [6]:
klikdokterlang = pd.read_csv(r'klikdokterfiltered.csv')

pattern = r'\bapp(s)?\b'
replacement = 'application'

def replace_applications(text):
    return re.sub(pattern, replacement, text, flags=re.IGNORECASE)

klikdokterlang['clean_emoji'] = klikdokterlang['content'].apply(lambda x: clean(x, no_emoji=True))
klikdokterlang['clean_emoji'] = klikdokterlang['clean_emoji'].apply(replace_applications)
klikdokterlang['clean_emoji'] = klikdokterlang['clean_emoji'].apply(lambda x: contractions.fix(x))

klikdokter_languages = []

for index, row in klikdokterlang.iterrows():
    try:
        language = detect(row['clean_emoji'])
    except:
        language = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['language'] = language
    klikdokter_languages.append(row)

klikdokterlang = pd.DataFrame(klikdokter_languages)

#print(klikdokterlang)

klikdokterlang['language'].value_counts()

id    586
en     72
no      5
tl      3
af      3
it      3
fr      3
et      1
sv      1
hr      1
so      1
es      1
cy      1
Name: language, dtype: int64

In [21]:
klikdokterlang.to_csv('klikdokterlang.csv',index=False)

# halodoc

In [10]:
reviews = reviews_all(
    'com.linkdokter.halodoc.android',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

halodocraw = pd.DataFrame(np.array(reviews),columns=['review'])
halodocraw = halodocraw.join(pd.DataFrame(halodocraw.pop('review').tolist()))
halodocraw['at'] = pd.to_datetime(halodocraw['at'], format='%Y-%m-%d')
halodocraw = halodocraw.loc[(halodocraw['at'] <= "2023-5-31")]
halodocraw

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
230,972c54ad-60ff-43fc-999c-54ff8372f668,INDRA SORANGAN,https://play-lh.googleusercontent.com/a/AAcHTt...,"2 kartu kredit ditolak, pake 1 lagi baru bisa,...",4,0,17.202,2023-05-30 20:03:49,Hai Indra Sorangan ! Mohon maaf atas ketidakny...,2023-05-30 20:08:23
231,ab688557-b8b2-4000-9e4b-f819ffc232f5,Fat_Kitten.0_0.,https://play-lh.googleusercontent.com/a-/AD_cM...,Aplikasi yg sangat membantu dan mempermudah ak...,5,0,None,2023-05-30 17:11:01,Hai Fat_Kitten.0_0! Terima kasih sudah menggun...,2023-05-30 17:22:33
232,e79eb59a-cf0b-42ad-8d37-3b7c5f683a07,Iin Yama,https://play-lh.googleusercontent.com/a/AAcHTt...,Cepat melayani 👍,5,0,17.202,2023-05-29 23:14:05,Hai Iin Yama! Terima kasih sudah menggunakan H...,2023-05-30 00:17:10
233,762ce30b-f602-4e5d-8cce-98b1e41a1993,Feizal Ramadani,https://play-lh.googleusercontent.com/a-/AD_cM...,Sangat membantu,5,0,None,2023-05-29 22:54:20,Hai Feizal Ramadani! Terima kasih sudah menggu...,2023-05-29 23:00:58
234,65974598-0b1a-4044-9872-f82ff606731b,Kartiwa Hardjanegara,https://play-lh.googleusercontent.com/a/AAcHTt...,Hari ini transaksi pakai Asuransi Umum Mega di...,4,0,17.202,2023-05-29 22:03:45,Hai Kartiwa Hardjanegara! Mohon maaf atas keti...,2023-05-29 21:52:24
...,...,...,...,...,...,...,...,...,...,...
34022,6be3f0f1-a715-4a4e-8d41-4c28a3dc0d51,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's very easy and helpful,5,2,None,2016-04-02 12:54:34,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:16:39
34023,15cad9f3-5d69-4b43-ba94-c5e788cf554b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Sepertinya ini aplikasi pertama di Indonesia.....,5,4,0.16.2-build201603301019,2016-04-02 11:06:15,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:19:05
34024,67a73c69-81af-4839-9fee-813b4b53bbfb,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Good aplication,5,1,0.16.2-build201603301019,2016-04-02 11:02:41,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:19:03
34025,61db862b-82b4-4c73-997b-b6c833595f74,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This is simplying health,5,1,0.13.1-build201603171038,2016-03-18 19:11:25,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:17:08


In [16]:
halodocraw.to_csv('halodocraw.csv',index=False)

In [11]:
halodocfiltered = halodocraw.copy()
halodocfiltered = halodocfiltered.reset_index(drop=True)
halodocfiltered = halodocfiltered.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','score'])
halodocfiltered = halodocfiltered[[len(i.split())>2 for i in halodocfiltered['content'].values]]
halodocfiltered.to_csv('halodocfiltered.csv',index=False)
halodocfiltered

,content
0,"2 kartu kredit ditolak, pake 1 lagi baru bisa,..."
1,Aplikasi yg sangat membantu dan mempermudah ak...
2,Cepat melayani 👍
4,Hari ini transaksi pakai Asuransi Umum Mega di...
5,Saran hilangkan pengiriman obat 3 hr
...,...
33791,It's much much easier to reach the doctor that...
33792,It's very easy and helpful
33793,Sepertinya ini aplikasi pertama di Indonesia.....
33795,This is simplying health


In [12]:
halodoclang = pd.read_csv(r'halodocfiltered.csv')

pattern = r'\bapp(s)?\b'
replacement = 'application'

def replace_applications(text):
    return re.sub(pattern, replacement, text, flags=re.IGNORECASE)

halodoclang['clean_emoji'] = halodoclang['content'].apply(lambda x: clean(x, no_emoji=True))
halodoclang['clean_emoji'] = halodoclang['clean_emoji'].apply(replace_applications)
halodoclang['clean_emoji'] = halodoclang['clean_emoji'].apply(lambda x: contractions.fix(x))

halodoc_languages = []

for index, row in halodoclang.iterrows():
    try:
        language = detect(row['clean_emoji'])
    except:
        language = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['language'] = language
    halodoc_languages.append(row)

halodoclang = pd.DataFrame(halodoc_languages)

#print(halodoclang)

halodoclang['language'].value_counts()

id    14827
en     5983
tl      268
fr      128
no      107
so       93
af       91
ca       80
da       75
it       73
et       65
ro       51
cy       38
nl       35
sl       28
sv       23
hr       22
es       20
fi       15
lt       15
sq       14
sk       11
tr       10
pl        7
cs        5
sw        5
pt        4
hu        4
de        2
lv        1
Name: language, dtype: int64

In [22]:
halodoclang.to_csv('halodoclang.csv',index=False)

# VADER

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [33]:
alodoktervader = pd.read_csv(r'alodokterlang.csv')
alodoktervader = alodoktervader.loc[alodoktervader['language'] == "en"]
alodoktervader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in alodoktervader['content']]
alodoktervader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in alodoktervader['content']]
alodoktervader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in alodoktervader['content']]
alodoktervader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in alodoktervader['content']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

alodoktervader['vader_rating'] = alodoktervader.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)
alodoktervader.to_csv('alodoktervader.csv',index=False)
alodoktervader

,content,clean_emoji,language,compound,neg,neu,pos,vader_rating
6,The most dissapointing things about this is th...,the most dissapointing things about this is th...,en,0.1524,0.047,0.922,0.032,1
23,This apps only indonesia can use ?,this application only indonesia can use ?,en,0.0000,0.000,1.000,0.000,0
29,bad app. u will verify your phone foerever! ve...,bad application. you will verify your phone fo...,en,-0.2242,0.267,0.524,0.209,-1
32,"Fast, reliable, cheap. Thank you.","fast, reliable, cheap. thank you.",en,0.3612,0.000,0.615,0.385,1
36,my experience consulting with doctor Dinar Wit...,my experience consulting with doctor dinar wit...,en,-0.6311,0.109,0.891,0.000,-1
...,...,...,...,...,...,...,...,...
10669,Great apps 👍👍👍,great application,en,0.6249,0.000,0.631,0.369,1
10686,Watta a good apps...congrats...n keep update !,watta a good application...congrats...n keep u...,en,0.4926,0.000,0.653,0.347,1
10693,"I love it, this app makes me easier to consult...","i love it, this application makes me easier to...",en,0.8020,0.098,0.543,0.359,1
10694,Finally alodokter launch the alodokter apps in...,finally alodokter launch the alodokter applica...,en,0.8591,0.000,0.708,0.292,1


In [34]:
klikdoktervader = pd.read_csv(r'klikdokterlang.csv')
klikdoktervader = klikdoktervader.loc[klikdoktervader['language'] == "en"]
klikdoktervader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in klikdoktervader['content']]
klikdoktervader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in klikdoktervader['content']]
klikdoktervader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in klikdoktervader['content']]
klikdoktervader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in klikdoktervader['content']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

klikdoktervader['vader_rating'] = klikdoktervader.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)
klikdoktervader.to_csv('klikdoktervader.csv',index=False)
klikdoktervader

,content,clean_emoji,language,compound,neg,neu,pos,vader_rating
19,Please fix your maps especially for list of ph...,please fix your maps especially for list of ph...,en,0.3182,0.000,0.777,0.223,1
136,"Skeptis sama app ini awalnya, abis baca review...","skeptis sama application ini awalnya, abis bac...",en,0.4939,0.000,0.814,0.186,1
182,Doctor end the conversation in the middle of c...,doctor end the conversation in the middle of c...,en,-0.2960,0.050,0.950,0.000,-1
273,You should Try now...,you should try now...,en,0.0000,0.000,1.000,0.000,0
337,"Worst app, dokter gak responsif","worst application, dokter gak responsif",en,-0.6249,0.506,0.494,0.000,-2
...,...,...,...,...,...,...,...,...
650,App dev is disgustingly unskilled.,application dev is disgustingly unskilled.,en,-0.5994,0.494,0.506,0.000,-1
662,"Very usefull aps, thx somuch..","very usefull aps, thanks somuch..",en,0.4144,0.000,0.591,0.409,1
678,"Very useful, complete info...thanks Dok","very useful, complete info...thanks dok",en,0.4927,0.000,0.556,0.444,1
679,I love the Tanya Dokter feature. Informative a...,i love the tanya dokter feature. informative a...,en,0.8494,0.000,0.593,0.407,1


In [36]:
halodocvader = pd.read_csv(r'halodoclang.csv')
halodocvader = halodocvader.loc[halodocvader['language'] == "en"]
halodocvader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in halodocvader['content']]
halodocvader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in halodocvader['content']]
halodocvader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in halodocvader['content']]
halodocvader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in halodocvader['content']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

halodocvader['vader_rating'] = halodocvader.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)
halodocvader.to_csv('halodocvader.csv',index=False)
halodocvader

,content,clean_emoji,language,compound,neg,neu,pos,vader_rating
14,Good app very helpful,good application very helpful,en,0.7178,0.000,0.250,0.750,2
19,"Very easy and usefull, and also it helps a lot...","very easy and usefull, and also it helps a lot...",en,0.8070,0.000,0.520,0.480,1
21,Good organize solve health problem very quick,good organize solve health problem very quick,en,0.2500,0.237,0.351,0.412,1
22,Very helpful when we need a doctor,very helpful when we need a doctor,en,0.4754,0.000,0.660,0.340,1
30,"The ""Call Now"" feature is very helpful if ther...","the ""call now"" feature is very helpful if ther...",en,-0.3197,0.240,0.622,0.138,-1
...,...,...,...,...,...,...,...,...
22092,Its easy to understand and have simple UI design,its easy to understand and have simple ui design,en,0.4404,0.000,0.734,0.266,1
22094,Like having own doctor,like having own doctor,en,0.3612,0.000,0.545,0.455,1
22095,It's much much easier to reach the doctor that...,it is much much easier to reach the doctor tha...,en,0.4404,0.000,0.755,0.245,1
22096,It's very easy and helpful,it is very easy and helpful,en,0.7397,0.000,0.324,0.676,2


# keybert

In [47]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\asus\appdata\local\programs\python\python39\lib\site-packages (1.2.0)



You should consider upgrading via the 'c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [64]:
from keybert import KeyBERT
kw_model = KeyBERT()

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words="english")

#noun noun, noun adj, noun verb, adj noun, adj verb, verb noun
#vectorizer = KeyphraseCountVectorizer(pos_pattern='<N.*>+<N.*>+|<N.*>+<J.*>+|<N.*>+<V.*>+|<J.*>+<N.*>+|<J.*>+<V.*>+|<V.*>*<N.*>+|')
#vectorizer = KeyphraseCountVectorizer()

TypeError: __init__() got an unexpected keyword argument 'use_maxsum'

In [72]:
alodokterkeybert = pd.read_csv(r'alodoktervader.csv')

alodokter_keyword = []
for index, row in alodokterkeybert.iterrows():
    try:
        keyword = kw_model.extract_keywords(row['clean_emoji'], vectorizer=vectorizer,use_mmr=True, diversity=0.7)
    except:
        keyword = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['keyword'] = keyword
    alodokter_keyword.append(row)

alodokterkeybert = pd.DataFrame(alodokter_keyword)
alodokterkeybert.to_csv('alodokterkeybert.csv',index=False)

In [73]:
klikdokterkeybert = pd.read_csv(r'klikdoktervader.csv')

klikdokter_keyword = []
for index, row in klikdokterkeybert.iterrows():
    try:
        keyword = kw_model.extract_keywords(row['clean_emoji'], vectorizer=vectorizer,use_mmr=True, diversity=0.7)
    except:
        keyword = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['keyword'] = keyword
    klikdokter_keyword.append(row)

klikdokterkeybert = pd.DataFrame(klikdokter_keyword)
klikdokterkeybert.to_csv('klikdokterkeybert.csv',index=False)

In [74]:
halodockeybert = pd.read_csv(r'halodocvader.csv')

halodoc_keyword = []
for index, row in halodockeybert.iterrows():
    try:
        keyword = kw_model.extract_keywords(row['clean_emoji'], vectorizer=vectorizer,use_mmr=True, diversity=0.7)
    except:
        keyword = "error"
        #print("This row (index {}) throws an error: {}".format(index, row['clean_emoji']))
    row['keyword'] = keyword
    halodoc_keyword.append(row)

halodockeybert = pd.DataFrame(halodoc_keyword)
halodockeybert.to_csv('halodockeybert.csv',index=False)

# explode

In [87]:
klikdokterkeybert = pd.read_csv(r'klikdokterkeybert.csv')
klikdokterkeybert = klikdokterkeybert.explode('keyword')
klikdokterkeybert['keyword'] = klikdokterkeybert['keyword'].str.extract(r"\('([^']*)',")
klikdokterkeybert.to_csv('klikdokterkeybertexploded.csv',index=False)

In [88]:
alodokterkeybert = pd.read_csv(r'alodokterkeybert.csv')
alodokterkeybert = alodokterkeybert.explode('keyword')
alodokterkeybert['keyword'] = alodokterkeybert['keyword'].str.extract(r"\('([^']*)',")
alodokterkeybert.to_csv('alodokterkeybertexploded.csv',index=False)

In [89]:
halodockeybert = pd.read_csv(r'halodockeybert.csv')
halodockeybert = halodockeybert.explode('keyword')
halodockeybert['keyword'] = halodockeybert['keyword'].str.extract(r"\('([^']*)',")
halodockeybert.to_csv('halodockeybertexploded.csv',index=False)